In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.orventro import deviation_metric, deviation_metric_optimized
from datetime import datetime

xgb.set_config(verbosity=0)

In [2]:
train_df = pd.read_csv('./data/train.csv', index_col='id')
target = 'per_square_meter_price'

/home/orventro/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def prep(df):
    df = df.drop(['city', 'osm_city_nearest_name', 'date'], axis=1)
    df = df.fillna(0.0)
    df = df[pd.to_numeric(df['floor'], errors='coerce').notnull()]
    df['floor'] = pd.to_numeric(df['floor'], errors='coerce')
    regions = df['region'].unique()
    reg_dict = dict(zip(regions, range(len(regions))))
    df['region'] = [reg_dict[r] for r in df['region']]
    df['street'] = [s if type(s) == float else float(s[1:]) for s in df['street']]
    return df.astype(float)

In [4]:
train_df = prep(train_df)
valid = train_df.iloc[100000:]
train = train_df.iloc[:100000]

In [5]:
def test_model1(model, train, valid):
    model.fit(train.drop([target], axis=1), train[target])
    pred = model.predict(valid.drop([target], axis=1))
    err = deviation_metric_optimized(pred, valid[target].to_numpy())
    print(model, err)

In [16]:
def train_stack(train):
    weights = train['price_type'].to_numpy() * 99 + 1
    model_cat = CatBoostRegressor(iterations=100, verbose=0,
                                 loss_function=deviation_metric_optimized,
                                 eval_metric=deviation_metric_optimized)
    model_xgb = XGBRegressor(device='GPU')
    model_lgb = LGBMRegressor(device='GPU')
    level0 = list()
    level0.append(('cat', model_cat))
    level0.append(('xgb', model_xgb))
    level0.append(('lgb', model_lgb))

    model_logreg = LinearRegression()

    model = StackingRegressor(estimators=level0, 
                              final_estimator=model_logreg, 
                              cv=5)
    model.fit(train.drop([target], axis=1), 
              train[target],
              sample_weight=weights)
    return model

def validate_stack(stack, valid):
    pred = stack.predict(valid.drop([target], axis=1))
    weights = valid['price_type'] * 99 + 1
    return deviation_metric_optimized(valid[target], pred, weights)

def predict(stack):
    test_df = pd.read_csv('./data/test.csv', index_col='id')
    test_df = prep(test_df)
    
    pred = pd.DataFrame(index=test_df.index)
    pred.insert(0, 'per_square_meter_price', stack.predict(test_df))
    pred.to_csv(f'./out_{datetime.now().time()}.csv')

In [17]:
stack = train_stack(train)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Got unsafe target value = 1.63931e+06 at object #576 of dataset learn


CatBoostError: catboost/python-package/catboost/helpers.cpp:42: Traceback (most recent call last):
  File "_catboost.pyx", line 1393, in _catboost._ObjectiveCalcDersRange
AttributeError: 'function' object has no attribute 'calc_ders_range'


In [8]:
err = validate_stack(stack, valid)
print(err)

3.012880936046989


In [9]:
predict(stack)